## <font color='Green'>Project Objective: Cross Platform Product Mapping Algorithm for Products
<hr style="border: 2px solid lightgray;">

###  <font color='blue'> Objective:
Build an algorithm to map product IDs of the same product across Qcomm (Quick Commerce) platforms.<br>    
**NOTE:**
*The following code is executable on Google Colaboratory.*


<font color='purple'>**Mounting G-Drive**

In [ ]:
# Mount G-Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<font color='blue'>**Defining necessary variables**

In [ ]:
## =================== DEFINING NECESSARY VARIABLES ===================================
# Dataset path
dataset_path = "/content/drive/MyDrive/Assignment_Data/Assignment_Data.xlsx" # EDIT IF REQUIRED

# Define minimal similarity threshold                                                # EDIT IF REQUIRED
similarity_threshold_ = 0.3    # CONSIDERING PRODUCTS THAT HAVE A MATCHING SIMILARITY SCORE HIGHER THAN THIS THRESHOLD
lev_threshold_ = 0.15  # CONSIDERING PRODUCTS THAT HAVE A MATCHING LEV THRESHOLD SCORE LESSER THAN THIS THRESHOLD

<font color='green'>**Importing necessary libraries and loading the datasets across all three Quick Commerce platforms (Blinkit, Zepto, Instamart)**

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Blinkit dataset (Dropping duplicates, if any)
blinkit_df = pd.read_excel(dataset_path, sheet_name='Blinkit').drop_duplicates().reset_index(drop=True)
# Handling cases where SAME PRODUCT exists across MULTIPLE CATEGORY (retaining one only)
blinkit_df = blinkit_df.drop_duplicates(subset=['product_id'], keep='first')

blinkit_df_orig = blinkit_df.copy()
print(f"Count of records in Blinkit dataset: {blinkit_df.shape[0]}")
print("Column names: ", list(blinkit_df.columns));print()

# Zepto dataset (Dropping duplicates, if any)
zepto_df = pd.read_excel(dataset_path, sheet_name='Zepto').drop_duplicates().reset_index(drop=True)
zepto_df_orig = zepto_df.copy()
# Handling cases where SAME PRODUCT exists across MULTIPLE CATEGORY (retaining one only)
zepto_df = zepto_df.drop_duplicates(subset=['product_id'], keep='first')
print(f"Count of records in Zepto dataset: {zepto_df.shape[0]}")
print("Column names: ", list(zepto_df.columns));print()


# Instamart dataset (Dropping duplicates, if any)
instamart_df = pd.read_excel(dataset_path, sheet_name='Instamart').drop_duplicates().reset_index(drop=True)
# Handling cases where SAME PRODUCT exists across MULTIPLE CATEGORY (retaining one only)
instamart_df = instamart_df.drop_duplicates(subset=['product_id'], keep='first')
instamart_df_orig = instamart_df.copy()
print(f"Count of records in Instamart dataset: {instamart_df.shape[0]}")
print("Column names: ", list(instamart_df.columns))


Count of records in Blinkit dataset: 3162
Column names:  ['gc_platform', 'product_id', 'Category', 'brand', 'title', 'grammage', 'mrp']

Count of records in Zepto dataset: 2625
Column names:  ['gc_platform', 'product_id', 'Category', 'brand', 'title', 'grammage', 'mrp']

Count of records in Instamart dataset: 2400
Column names:  ['gc_platform', 'product_id', 'Category', 'brand', 'title', 'grammage', 'mrp']


In [ ]:
# -----------------------------------------------------------
# Preliminary Data Pre-processing
# -----------------------------------------------------------

# Defining a function to rename columns in dataframes across all the three gc_platforms
def rename_cols_gc_platforms(df):
  df = df.rename(columns={'gc_platform': 'PLATFORM',
                            'product_id': 'PRODUCT_ID',
                            'Category': 'CATEGORY',
                            'brand': 'BRAND',
                            'title': 'TITLE',
                            'grammage': 'VOLUME_SIZE',
                            'mrp': 'MRP'})
  # Reordering columns, sorting dataframe and resetting indices
  df = df.loc[:, ['PLATFORM', 'PRODUCT_ID', 'TITLE', 'CATEGORY', 'BRAND', 'VOLUME_SIZE', 'MRP']]
  return df.sort_values(by='PRODUCT_ID', ascending=True).reset_index(drop=True)

# Dataframes with columns renamed
blinkit_df = rename_cols_gc_platforms(blinkit_df)
zepto_df = rename_cols_gc_platforms(zepto_df)
instamart_df = rename_cols_gc_platforms(instamart_df)


In [ ]:
# Display records from the blinkit dataset
blinkit_df.head(5)

,PLATFORM,PRODUCT_ID,TITLE,CATEGORY,BRAND,VOLUME_SIZE,MRP
0,Blinkit,56,MDH Chunky Chat Masala,"Masalas, Oils & Sugar",MDH,100 g,86.0
1,Blinkit,57,Everest Chat Masala,"Masalas, Oils & Sugar",Everest,50 g,42.0
2,Blinkit,58,MDH Chicken Masala,"Masalas, Oils & Sugar",MDH,100 g,87.0
3,Blinkit,59,Everest Chicken Masala,"Masalas, Oils & Sugar",Everest,100 g,92.0
4,Blinkit,61,Everest Chole Masala,"Masalas, Oils & Sugar",Everest,100 g,88.0


In [ ]:
# Checking datatypes
blinkit_df.dtypes

,0
PLATFORM,object
PRODUCT_ID,int64
TITLE,object
CATEGORY,object
BRAND,object
VOLUME_SIZE,object
MRP,float64


In [ ]:
# Checking datatypes
blinkit_df.describe()

,PRODUCT_ID,MRP
count,3162.000000,3060.000000
mean,406330.880772,142.651961
std,180699.186324,135.547333
min,56.000000,5.000000
25%,376823.000000,57.000000
50%,485203.500000,96.000000
75%,532382.250000,181.250000
max,579629.000000,1200.000000


In [ ]:
# Exploring unique values in 'CATEGORY'
blinkit_df['VOLUME_SIZE'].value_counts()

,count
VOLUME_SIZE,
100 g,736
50 g,379
200 g,270
500 g,197
100 ml,93
...,...
1 pack (25 Sachets),1
151.2 g,1
145 g,1


**Observations:**<br>Different representations, including the units of solid and liquid items, need to be standardized into a specific format to enable meaningful comparison of products across the data from the Zepto and Instamart platforms.

In [ ]:
# Exploring unique values in 'CATEGORY'
blinkit_df['TITLE'].value_counts()

,count
TITLE,
Sugar Free Gold Sweetener,5
Sugar Free Natura Sweetener,4
Whole Farm Premium Red Chilli Powder,3
Vandevi 2001 Hing Powder Brown,3
Whole Farm Premium Coriander Powder,3
...,...
BitMo Tooty Fruity Bakery Cookies,1
BitMo Jeera Bakery Cookies,1
BitMo Ajwain Bakery Cookies,1


**Observations:**<br>Most of the titles are inherently distinct when considering the distinct count. Therefore, we should assign greater weight to `'TITLE'`.

In [ ]:
# Checking count of null values across all columns
blinkit_df.isnull().sum()

,0
PLATFORM,0
PRODUCT_ID,0
TITLE,0
CATEGORY,0
BRAND,6
VOLUME_SIZE,0
MRP,102


In [ ]:
# Checking count of null values across all columns
zepto_df.isnull().sum()

,0
PLATFORM,0
PRODUCT_ID,0
TITLE,0
CATEGORY,0
BRAND,1
VOLUME_SIZE,0
MRP,41


In [ ]:
# Checking count of null values across all columns
instamart_df.isnull().sum()

,0
PLATFORM,0
PRODUCT_ID,0
TITLE,0
CATEGORY,0
BRAND,6
VOLUME_SIZE,0
MRP,9


**Observations:**<br>Missing values do exist in `'BRAND'` and `'MRP'` columns.

In [ ]:
import pandas as pd
import re
from sklearn.preprocessing import MinMaxScaler

# ---------------------------------------------------------------------
# Preliminary: Text Normalization and Cleaning
# ---------------------------------------------------------------------
def normalize_text(text):
    text = str(text).lower()
    text = re.sub(r"[^\w\s]", "", text)  # Remove special characters
    text = text.strip()  # Remove extra spaces
    return text

def return_text_normalized_n_imputed_df(df):
  df["TITLE"] = df["TITLE"].apply(normalize_text).fillna("Unknown")
  df["CATEGORY"] = df["CATEGORY"].apply(normalize_text).fillna("Unknown")
  df["BRAND"] = df["BRAND"].apply(normalize_text).fillna("Unknown")
  df["VOLUME_SIZE"] = df["VOLUME_SIZE"].apply(normalize_text).fillna("Unknown")
  return df


# Text Normalization and Cleaning (across all 3 dataframes)
blinkit_df = return_text_normalized_n_imputed_df(blinkit_df)
zepto_df = return_text_normalized_n_imputed_df(zepto_df)
instamart_df = return_text_normalized_n_imputed_df(instamart_df)

In [ ]:
blinkit_df.head(10)

,PLATFORM,PRODUCT_ID,TITLE,CATEGORY,BRAND,VOLUME_SIZE,MRP
0,Blinkit,56,mdh chunky chat masala,masalas oils sugar,mdh,100 g,86.0
1,Blinkit,57,everest chat masala,masalas oils sugar,everest,50 g,42.0
2,Blinkit,58,mdh chicken masala,masalas oils sugar,mdh,100 g,87.0
3,Blinkit,59,everest chicken masala,masalas oils sugar,everest,100 g,92.0
4,Blinkit,61,everest chole masala,masalas oils sugar,everest,100 g,88.0
5,Blinkit,62,mdh jeera powder cumin powder,masalas oils sugar,mdh,100 g,92.0
6,Blinkit,63,everest jeera powder cumin powder,masalas oils sugar,everest,100 g,102.0
7,Blinkit,64,mdh makhani dal masala,masalas oils sugar,mdh,100 g,75.0
8,Blinkit,65,everest ginger powder,masalas oils sugar,everest,100 g,90.0
9,Blinkit,66,mdh amchur powder,masalas oils sugar,mdh,100 g,90.0


In [ ]:
# Rechecking count of null values across all columns
blinkit_df.isnull().sum()

,0
PLATFORM,0
PRODUCT_ID,0
TITLE,0
CATEGORY,0
BRAND,0
VOLUME_SIZE,0
MRP,102


**Observation:** Missing values still exist in the `'MRP'` column. Since these are numerical, they should be addressed separately using a context-based approach.

In [ ]:
# Defining a strategy for imputing missing values in 'MRP' columns across all 3 datasets

def imputing_mrp_missing_vals_df(df):
    """
    Imputes missing values in the 'MRP' column of a DataFrame based on grouping by 'CATEGORY' and 'BRAND'.

    This function fills missing 'MRP' values in two steps:
    1. First, it calculates the median MRP for each unique combination of 'CATEGORY' and 'BRAND' and uses it
       to fill missing MRP values within those groups.
    2. If any missing values remain (e.g., for rows with unknown brands), it calculates the median MRP for
       each 'CATEGORY' and fills the remaining missing values based on the category median.

    Parameters:
        df (pd.DataFrame): A pandas DataFrame containing at least the columns 'MRP', 'CATEGORY', and 'BRAND'.

    Returns:
        pd.DataFrame: The input DataFrame with missing 'MRP' values imputed.
    """

    # Fill missing MRP values with the median MRP of their CATEGORY and BRAND group
    df["MRP"] = df.groupby(["CATEGORY", "BRAND"])["MRP"].transform(lambda x: x.fillna(x.median()))

    # If there are still missing values (e.g., for unknown brands), use the CATEGORY median
    df["MRP"] = df.groupby("CATEGORY")["MRP"].transform(lambda x: x.fillna(x.median()))

    return df

# Imputing missing values in dataframes across all 3 platforms
blinkit_df = imputing_mrp_missing_vals_df(blinkit_df)
zepto_df = imputing_mrp_missing_vals_df(zepto_df)
instamart_df = imputing_mrp_missing_vals_df(instamart_df)


In [ ]:
blinkit_df.head(10)

,PLATFORM,PRODUCT_ID,TITLE,CATEGORY,BRAND,VOLUME_SIZE,MRP
0,Blinkit,56,mdh chunky chat masala,masalas oils sugar,mdh,100 g,86.0
1,Blinkit,57,everest chat masala,masalas oils sugar,everest,50 g,42.0
2,Blinkit,58,mdh chicken masala,masalas oils sugar,mdh,100 g,87.0
3,Blinkit,59,everest chicken masala,masalas oils sugar,everest,100 g,92.0
4,Blinkit,61,everest chole masala,masalas oils sugar,everest,100 g,88.0
5,Blinkit,62,mdh jeera powder cumin powder,masalas oils sugar,mdh,100 g,92.0
6,Blinkit,63,everest jeera powder cumin powder,masalas oils sugar,everest,100 g,102.0
7,Blinkit,64,mdh makhani dal masala,masalas oils sugar,mdh,100 g,75.0
8,Blinkit,65,everest ginger powder,masalas oils sugar,everest,100 g,90.0
9,Blinkit,66,mdh amchur powder,masalas oils sugar,mdh,100 g,90.0


In [ ]:
# Recheck: No null values should exist
blinkit_df.isnull().sum()

,0
PLATFORM,0
PRODUCT_ID,0
TITLE,0
CATEGORY,0
BRAND,0
VOLUME_SIZE,0
MRP,0


**Standardizing the values in `'VOLUME_SIZE'`.**

In [ ]:
blinkit_df.VOLUME_SIZE.value_counts()

,count
VOLUME_SIZE,
100 g,736
50 g,379
200 g,270
500 g,197
100 ml,93
...,...
1512 g,1
145 g,1
145 g 4 pieces,1


In [ ]:
# Function to standardize volume size (grams, milliliters, etc.)
def convert_to_standardized_volume(volume):
    """
    Convert a volume string to a standardized numerical representation.

    This function processes and standardizes volume descriptions (e.g., "100 grams", "1 kg", "2 x 100 ml")
    to return a consistent numerical value. It handles various unit formats and conversions
    to ensure uniformity in data representation.

    Parameters:
        volume (str): A string describing the volume, which may include units like grams (g), kilograms (kg),
                      milliliters (ml), liters (l), or multi-part descriptions (e.g., "2 x 100 ml").

    Returns:
        float or str: The standardized numerical value in grams or milliliters, or the original string if
                      the volume format is unrecognized.

    Processing:
    1. Normalize unit variations:
      - Converts variations of "grams" (e.g., "gms", "grams") to "g".
      - Converts variations of "milliliters" (e.g., "mls") to "ml".
      - Ensures consistent formats for "kg" (kilograms) and "l" (liters).

    2. Unit conversions:
      - Converts kilograms (kg) to grams (g) by multiplying by 1000.
      - Converts liters (l) to milliliters (ml) by multiplying by 1000.

    3. Handles multi-part volume descriptions:
      - Parses and calculates volumes for formats like "4 x 100 g" or "1 pack of 12 x 500 ml".

    4. Handles "pieces" or "units":
      - Extracts the number of pieces or units where applicable.

    5. Returns unrecognized volume formats as-is:
      - If the format is unrecognized, the original volume string is returned.

    Examples:
        >>> convert_to_standardized_volume("1 kg")
        1000.0
        >>> convert_to_standardized_volume("2 x 500 ml")
        1000.0
        >>> convert_to_standardized_volume("200 g 4 pieces")
        200.0
        >>> convert_to_standardized_volume("unknown format")
        'unknown format'
    """

    # Normalize variations in grams (e.g., gms, gram, g)
    volume = volume.lower()
    volume = re.sub(r'\b(gms?|grams?|g)\b', 'g', volume)  # Convert all variations of grams to 'g'
    volume = re.sub(r'\bmls?\b', 'ml', volume)  # Normalize ml/MLS to ml
    volume = re.sub(r'\bkg\b', 'kg', volume)  # Ensure we have 'kg' for kilograms
    volume = re.sub(r'\bl\b', 'l', volume)  # Ensure we have 'l' for liters

    # Convert kg to g (1 kg = 1000 g)
    match_kg = re.match(r'(\d+(?:\.\d+)?)\s*kg', volume)
    if match_kg:
        value = float(match_kg.group(1)) * 1000  # Multiply by 1000 to convert kg to g
        return value

    # Convert L to ml (1 L = 1000 ml)
    match_L = re.match(r'(\d+(?:\.\d+)?)\s*l', volume)
    if match_L:
        value = float(match_L.group(1)) * 1000  # Multiply by 1000 to convert L to ml
        return value

    # Handle basic grams and milliliters
    match = re.match(r'(\d+(?:\.\d+)?)\s*(g|ml)', volume)
    if match:
        value, unit = match.groups()
        return float(value) if unit == 'g' else float(value)  # For ml, just return the same value as grams

    # Handle multi-part descriptions (e.g., "1 x 1 pack 12 x 5 g")
    match_multi = re.match(r'(\d+)\s*x\s*(\d+)\s*pack\s*(\d+)\s*x\s*(\d+(?:\.\d+)?)\s*(g|ml)', volume)
    if match_multi:
        quantity, packs, units, unit_value, unit_type = match_multi.groups()
        total = int(quantity) * int(packs) * int(units) * float(unit_value)
        return total if unit_type == 'g' else total  # Adjust for ml if needed

    # Handle pieces or units (e.g., "145 g 4 pieces" or "1 piece")
    match_pieces = re.match(r'(\d+(?:\.\d+)?)\s*(g|ml)\s*(\d+)\s*(pieces|units)', volume)
    if match_pieces:
        value, unit, pieces, _ = match_pieces.groups()
        return float(value) if unit == 'g' else float(value)  # Handle accordingly

    # Handle single pieces or units (e.g., "1 piece", "1 unit")
    match_single = re.match(r'(\d+)\s*(pieces|piece|unit|units)', volume)
    if match_single:
        return 1  # Treat "1 piece", "1 unit", "200 units", etc., as a standard volume of 1 unit

    # Handle cases like "4 x 100 ml"
    match_x_ml = re.match(r'(\d+)\s*x\s*(\d+(?:\.\d+)?)\s*(ml|g)', volume)
    if match_x_ml:
        quantity, value, unit = match_x_ml.groups()
        total = int(quantity) * float(value)
        return total if unit == 'g' else total  # Adjust for ml if needed

    # For unrecognized units (e.g., "100 pallets"), just return the numerical value
    match_unrecognized = re.match(r'(\d+(?:\.\d+)?)\s+\w+', volume)
    if match_unrecognized:
        return float(match_unrecognized.group(1))

    # If no match, return the original volume size as it is
    return volume

# Apply the conversion function to the 'VOLUME_SIZE' column across all 3 datasets
blinkit_df['VOLUME_SIZE'] = blinkit_df['VOLUME_SIZE'].apply(convert_to_standardized_volume).astype(str)
zepto_df['VOLUME_SIZE'] = zepto_df['VOLUME_SIZE'].apply(convert_to_standardized_volume).astype(str)
instamart_df['VOLUME_SIZE'] = instamart_df['VOLUME_SIZE'].apply(convert_to_standardized_volume).astype(str)


In [ ]:
blinkit_df.head(10)

,PLATFORM,PRODUCT_ID,TITLE,CATEGORY,BRAND,VOLUME_SIZE,MRP
0,Blinkit,56,mdh chunky chat masala,masalas oils sugar,mdh,100.0,86.0
1,Blinkit,57,everest chat masala,masalas oils sugar,everest,50.0,42.0
2,Blinkit,58,mdh chicken masala,masalas oils sugar,mdh,100.0,87.0
3,Blinkit,59,everest chicken masala,masalas oils sugar,everest,100.0,92.0
4,Blinkit,61,everest chole masala,masalas oils sugar,everest,100.0,88.0
5,Blinkit,62,mdh jeera powder cumin powder,masalas oils sugar,mdh,100.0,92.0
6,Blinkit,63,everest jeera powder cumin powder,masalas oils sugar,everest,100.0,102.0
7,Blinkit,64,mdh makhani dal masala,masalas oils sugar,mdh,100.0,75.0
8,Blinkit,65,everest ginger powder,masalas oils sugar,everest,100.0,90.0
9,Blinkit,66,mdh amchur powder,masalas oils sugar,mdh,100.0,90.0


In [ ]:
blinkit_df.tail(10)

,PLATFORM,PRODUCT_ID,TITLE,CATEGORY,BRAND,VOLUME_SIZE,MRP
3152,Blinkit,571731,cookieman celebrations diwali gift box,biscuits cookies cakes,cookieman,318.0,499.0
3153,Blinkit,572291,sri vasavi condiments home made sugar batasha,masalas oils sugar,sri vasavi condiments,150.0,60.0
3154,Blinkit,572972,havmor maharaja rajbhog matka kulfi,ice cream frozen desserts,havmor,500.0,300.0
3155,Blinkit,573995,havmor choco brownie ice cream tub pack of 3,ice cream frozen desserts,havmor,2250.0,750.0
3156,Blinkit,575643,whole farm premium khand,masalas oils sugar,whole farm,500.0,56.0
3157,Blinkit,575672,verduijns chocolate thins biscuit,biscuits cookies cakes,verduijns,75.0,390.0
3158,Blinkit,575674,verduijns mustard wafers,biscuits cookies cakes,verduijns,75.0,390.0
3159,Blinkit,576060,cream bell chocolate ice cream cup no added su...,ice cream frozen desserts,cream bell,400.0,70.0
3160,Blinkit,577044,verduijns fine biscuits traditionally baked bu...,biscuits cookies cakes,verduijns,75.0,390.0
3161,Blinkit,579629,orika omelette bhurji masala instant mix buy...,masalas oils sugar,orika,60.0,120.0


In [ ]:
blinkit_df['VOLUME_SIZE'].value_counts()

,count
VOLUME_SIZE,
100.0,835
50.0,381
200.0,281
500.0,264
1000.0,115
...,...
3575.0,1
215.0,1
55405.0,1


In [ ]:
!pip install faiss-cpu

<!-- The following method for mapping product IDs between datasets using **Cosine Similarity** with FAISS.

Key Highlights:
1. **Weighted TF-IDF Vectorization**:
   - Textual features from multiple columns are converted into weighted TF-IDF vectors.
   - Importance weights are applied to prioritize specific columns, such as TITLE or BRAND.

2. **Cosine Similarity with FAISS**:
   - Vectors are normalized to enable cosine similarity computation.
   - The FAISS `IndexFlatIP` is used for efficient similarity searches, leveraging inner product similarity.

3. **Threshold-Based Mapping**:
   - Products from one dataset are matched to the most similar entries in another dataset.
   - A user-defined similarity threshold ensures only sufficiently similar matches are considered.

### Steps:
- **Preprocessing**:
  Combine text from selected columns, apply TF-IDF vectorization, and normalize vectors for cosine similarity.
  
- **Similarity Search**:
  Use FAISS for fast nearest neighbor search based on cosine similarity.
  
- **Result Mapping**:
  Identify the closest product in the reference dataset for each entry in the target dataset, filtering by a similarity threshold.

### Applications:
- Product catalog matching between platforms (e.g., Blinkit and Zepto).
- Entity resolution tasks where textual similarity plays a key role.

### Example:
The algorithm maps product IDs from a source dataset to a target dataset with a similarity threshold (e.g., 0.5). Products that don't meet the threshold are marked as 'NOT FOUND'. -->


## <font color='green'> **Processes in the Approach**
### <font color='purple'>1. **Necessary Libraries**
   - `numpy`: For numerical operations, especially for array manipulation and vector normalization.
   - `pandas`: For data manipulation and dataframe operations.
   - `faiss`: For efficient similarity search based on vectorized representations.
   - `sklearn.feature_extraction.text.TfidfVectorizer`: For converting text data into numerical vectors using TF-IDF.
   - `Levenshtein`: For calculating Levenshtein distance to compare string similarities.

### <font color='purple'>2. **Text to Vector Conversion**
   - The function `text_to_vector`:
     - Takes two dataframes (`df1`, `df2`), columns to convert (`columns`), and their associated weights (`weights`).
     - Concatenates the specified columns and applies weights to create a weighted textual representation.
     - Uses the `TfidfVectorizer` to fit the concatenated text and then transforms the individual datasets (`df1`, `df2`) into TF-IDF vectors.
     - Normalizes the vectors for cosine similarity.

### <font color='purple'>3. **Cosine Similarity Mapping**
   - The function `return_mapped_ids`:
     - Converts text data into vectors using `text_to_vector`.
     - Performs a similarity search using FAISS (Fast Approximate Nearest Neighbor Search) based on cosine similarity.
     - Maps the product IDs between `df1` and `df2` by comparing the cosine similarity scores with a given threshold. If similarity is above the threshold, it maps the IDs; otherwise, it assigns `'NOT FOUND'`.

### <font color='purple'>4. **Levenshtein Distance Calculation**
   - The function `calculate_levenshtein_similarity`:
     - Calculates the Levenshtein distance (edit distance) between two strings and returns a similarity score.
     - The score is normalized by the maximum length of the two strings to avoid division by zero.

### <font color='purple'>5. **Processing Titles**
   - The function `process_titles_test_df`:
     - Merges the `test_df` with the `blinkit_df` to add corresponding Blinkit product titles based on the previously mapped product IDs.
     - Converts all titles to strings, replacing any NaN values with empty strings.
     - Uses the Levenshtein distance to compute similarity between `TITLE` and `BLINKIT_TITLE` for each entry.
     - Filters out titles with a similarity score below a given threshold (defined by `threshold`).
     - Updates the `MAPPED_BLINKIT_PRODUCT_ID` column based on the similarity results and drops irrelevant columns.

### <font color='purple'>6. **Processing the DataFrames**
   - After the similarity mappings are performed using cosine similarity in the `return_mapped_ids` function, the product IDs from the Blinkit dataset (`blinkit_df`) are mapped to Zepto (`zepto_df`) and Instamart (`instamart_df`) datasets using the `process_titles_test_df` function.
   - The Levenshtein similarity filter is applied to refine the product ID mappings, especially for entries that didn't match in the initial cosine similarity search.


In [ ]:
import numpy as np
import pandas as pd
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

def text_to_vector(df1, df2, columns, weights):
    """
    Convert textual columns to TF-IDF vectors using the same vocabulary for both datasets,
    with feature weighting applied to the columns.
    columns: List of column names to be used
    weights: List of weights corresponding to the importance of each column
    """
    assert len(columns) == len(weights), "The number of columns must match the number of weights"

    # Create weighted text data
    weighted_text = pd.concat([df1[columns], df2[columns]]).apply(
        lambda row: ' '.join([f"{str(text)} " * weight for text, weight in zip(row, weights)]), axis=1)

    # Fit the vectorizer
    vectorizer.fit(weighted_text)

    # Convert datasets to TF-IDF vectors
    df1_vectors = vectorizer.transform(df1[columns].apply(lambda x: ' '.join([str(i) for i in x]), axis=1)).toarray().astype('float32')
    df2_vectors = vectorizer.transform(df2[columns].apply(lambda x: ' '.join([str(i) for i in x]), axis=1)).toarray().astype('float32')

    # Normalize vectors for cosine similarity
    df1_vectors /= np.linalg.norm(df1_vectors, axis=1, keepdims=True)
    df2_vectors /= np.linalg.norm(df2_vectors, axis=1, keepdims=True)

    return df1_vectors, df2_vectors

def return_mapped_ids(df1, df2, similarity_threshold=0.5):
    """
    Maps product IDs from one dataset to another using Cosine Similarity.
    similarity_threshold: Minimum cosine similarity score to consider a match
    """
    weights = [25, 4, 15, 3, 3]  # Adjust weights based on importance
    df1_vectors, df2_vectors = text_to_vector(df1, df2, ['TITLE', 'CATEGORY', 'BRAND', 'VOLUME_SIZE', 'MRP'], weights)

    # Step 2: Perform FAISS-based similarity search with Cosine similarity
    index = faiss.IndexFlatIP(df1_vectors.shape[1])  # Inner Product similarity
    index.add(df1_vectors)  # Add normalized df1 vectors to the FAISS index

    # Search for the nearest neighbors in df1 for each df2 entry
    D, I = index.search(df2_vectors, 1)  # Search for 1 nearest neighbor

    # Step 3: Map results based on similarity threshold
    mapped_ids = []
    for similarity, idx in zip(D, I):
        if similarity[0] >= similarity_threshold:  # Higher similarity indicates a better match
            mapped_ids.append(df1.iloc[idx[0]]['PRODUCT_ID'])
        else:
            mapped_ids.append('NOT FOUND')

    df2['MAPPED_BLINKIT_PRODUCT_ID'] = mapped_ids
    return df2

# Example with a similarity threshold
# EDIT SIMILARITY THRESHOLD (IF REQUIRED)
zepto_df = return_mapped_ids(df1=blinkit_df, df2=zepto_df, similarity_threshold=similarity_threshold_)
instamart_df = return_mapped_ids(df1=blinkit_df, df2=instamart_df, similarity_threshold=similarity_threshold_)


In [ ]:
zepto_df.head(10)

,PLATFORM,PRODUCT_ID,TITLE,CATEGORY,BRAND,VOLUME_SIZE,MRP,MAPPED_BLINKIT_PRODUCT_ID
0,Zepto,00243799-0502-4fa8-853b-fe3ad78ae09f,dukes bourbon biscuits,biscuits cookies cakes,dukes,135.0,36.0,424202
1,Zepto,0045eb3e-05b0-4c7a-8ec7-92d9ac6d5955,britannia cake roll yo swiss roll triple choc...,biscuits cookies cakes,britannia cakes,165.0,90.0,507447
2,Zepto,0046fb0f-391a-4447-8930-c35ba2dabbb4,equal stevia natural sweetener sugar free diab...,masalas oils sugar,equal,1,120.0,423328
3,Zepto,004e00e4-aba2-4387-88a0-d0e6423e543b,theee mango rasam powder,masalas oils sugar,three mango,100.0,60.0,349134
4,Zepto,0099fe94-ea23-4106-87b5-a6aaaa7520c8,go zero zero sugar nostalgia mango duet 60 m...,ice cream frozen desserts,bundle,4.0,500.0,538872
5,Zepto,009ffcf4-9972-4b90-b36b-9f689a2d1795,britannia treat jimjam cream biscuits,biscuits cookies cakes,britannia,460.0,100.0,59811
6,Zepto,00a59209-0c27-4280-b42e-38de8ace24d1,on1y garlic herb bread seasoning,masalas oils sugar,on1y,48.0,99.0,475790
7,Zepto,00abf5ff-297d-4e4e-8f03-6579121f0577,dnv dry red chilly whole,masalas oils sugar,dnv,50.0,30.0,NOT FOUND
8,Zepto,0133c4fa-274c-4d29-97b5-10b3a609e8ca,hocco chocaboom bar,ice cream frozen desserts,hocco ice creams,80.0,45.0,549416
9,Zepto,0134c33b-bdb2-45b9-8074-b817aed940be,mayurank elaichi small premium,masalas oils sugar,mayurank,50.0,259.0,NOT FOUND


In [ ]:
!pip install python-Levenshtein
import pandas as pd
from Levenshtein import distance as lev_distance
import numpy as np

# Levenshtein Distance Function (optimized for vectorized operation)
def calculate_levenshtein_similarity(str1, str2):
    lev_dist = lev_distance(str1, str2)
    max_len = max(len(str1), len(str2))
    return lev_dist / max_len if max_len > 0 else 0  # Avoid division by zero

# Function to process the DataFrame in a vectorized manner
def process_titles_test_df(blinkit_df, test_df, threshold=0.3):

    df = test_df.copy()

    ### Fetching Zepto and Blinkit titles in one dataframe
    df = df.merge(blinkit_df.loc[:,['PRODUCT_ID', 'TITLE']].rename(columns={'PRODUCT_ID': 'BLINKIT_PRODUCT_ID',\
                                                                                    'TITLE': 'BLINKIT_TITLE'}),
                              left_on='MAPPED_BLINKIT_PRODUCT_ID', right_on='BLINKIT_PRODUCT_ID', how='left') # IMP: Left join (to retain NOT FOUND)
    df.drop('BLINKIT_PRODUCT_ID', axis=1, inplace=True)

    # Ensure columns are strings (replace NaN/None with empty string)
    df['TITLE'] = df['TITLE'].fillna('').astype(str)
    df['BLINKIT_TITLE'] = df['BLINKIT_TITLE'].fillna('').astype(str)

    # Vectorized approach: mask where 'BLINKIT_TITLE' is 'NOT FOUND'
    mask_not_found = df['BLINKIT_TITLE'] == 'NOT FOUND'

    # Apply Levenshtein similarity only where it's not 'NOT FOUND'
    similarities = np.vectorize(calculate_levenshtein_similarity)(
        df['TITLE'].values, df['BLINKIT_TITLE'].values
    )

    # Create the 'NEW_TITLE' column with conditions
    df['NEW_MAPPED_BLINKIT_PRODUCT_ID'] = np.where(mask_not_found, 'NOT FOUND',
                               np.where(similarities < threshold, df['MAPPED_BLINKIT_PRODUCT_ID'], 'NOT FOUND'))

    # Dropping irrelevant columns
    df.drop(['MAPPED_BLINKIT_PRODUCT_ID', 'BLINKIT_TITLE'], axis=1, inplace=True)

    # Rename newly mapped blinkit product id
    df.rename(columns={'NEW_MAPPED_BLINKIT_PRODUCT_ID': 'MAPPED_BLINKIT_PRODUCT_ID'}, inplace=True)

    return df

# Eliminating product id's post computing levenshtein similarity for qualified items from cosine similarity filter
zepto_df = process_titles_test_df(blinkit_df, test_df=zepto_df, threshold=lev_threshold_)
instamart_df = process_titles_test_df(blinkit_df, test_df=instamart_df, threshold=lev_threshold_)

In [ ]:
zepto_df.head(10)

,PLATFORM,PRODUCT_ID,TITLE,CATEGORY,BRAND,VOLUME_SIZE,MRP,MAPPED_BLINKIT_PRODUCT_ID
0,Zepto,00243799-0502-4fa8-853b-fe3ad78ae09f,dukes bourbon biscuits,biscuits cookies cakes,dukes,135.0,36.0,424202
1,Zepto,0045eb3e-05b0-4c7a-8ec7-92d9ac6d5955,britannia cake roll yo swiss roll triple choc...,biscuits cookies cakes,britannia cakes,165.0,90.0,NOT FOUND
2,Zepto,0046fb0f-391a-4447-8930-c35ba2dabbb4,equal stevia natural sweetener sugar free diab...,masalas oils sugar,equal,1,120.0,NOT FOUND
3,Zepto,004e00e4-aba2-4387-88a0-d0e6423e543b,theee mango rasam powder,masalas oils sugar,three mango,100.0,60.0,349134
4,Zepto,0099fe94-ea23-4106-87b5-a6aaaa7520c8,go zero zero sugar nostalgia mango duet 60 m...,ice cream frozen desserts,bundle,4.0,500.0,NOT FOUND
5,Zepto,009ffcf4-9972-4b90-b36b-9f689a2d1795,britannia treat jimjam cream biscuits,biscuits cookies cakes,britannia,460.0,100.0,NOT FOUND
6,Zepto,00a59209-0c27-4280-b42e-38de8ace24d1,on1y garlic herb bread seasoning,masalas oils sugar,on1y,48.0,99.0,NOT FOUND
7,Zepto,00abf5ff-297d-4e4e-8f03-6579121f0577,dnv dry red chilly whole,masalas oils sugar,dnv,50.0,30.0,NOT FOUND
8,Zepto,0133c4fa-274c-4d29-97b5-10b3a609e8ca,hocco chocaboom bar,ice cream frozen desserts,hocco ice creams,80.0,45.0,NOT FOUND
9,Zepto,0134c33b-bdb2-45b9-8074-b817aed940be,mayurank elaichi small premium,masalas oils sugar,mayurank,50.0,259.0,NOT FOUND


### <font color='green'>**Handling cases where the VOLUME/SIZE could be different.**

### <font color='purple'>**Assuming that the MRP for products of the same weight varies within a range of up to 20% of the matched product.**

In [ ]:
import numpy as np
import pandas as pd

def return_mrp_vals_handled_test_df(original_df, test_df):
    """
    This function processes a test DataFrame by mapping the Maximum Retail Price (MRP) of products
    from the Blinkit platform (original_df) to corresponding products in the test DataFrame.
    It checks if the MRP values are within a 20% range of each other. If they are not, it updates
    the MAPPED_BLINKIT_PRODUCT_ID to "NOT FOUND".

    Parameters:
    original_df (DataFrame): A DataFrame containing the Blinkit platform dataset with PRODUCT_ID and MRP.
    test_df (DataFrame): A DataFrame containing the test product data with MAPPED_BLINKIT_PRODUCT_ID
                         and MRP (of test_df products) columns.

    Returns:
    DataFrame: The updated test DataFrame with modified MAPPED_BLINKIT_PRODUCT_ID values based on
                the MRP comparison.
    """

    # Merge the original Blinkit DataFrame with the test DataFrame to get the MRP values for Blinkit products
    test_df = test_df.merge(
        original_df.loc[:, ['PRODUCT_ID', 'MRP']].rename(columns={'PRODUCT_ID': 'BLINKIT_PRODUCT_ID', 'MRP': 'BLINKIT_MRP'}),
        how='left',
        left_on='MAPPED_BLINKIT_PRODUCT_ID',
        right_on='BLINKIT_PRODUCT_ID'
    ).drop('BLINKIT_PRODUCT_ID', axis=1)

    # Create a mask for rows where MAPPED_BLINKIT_PRODUCT_ID is not "NOT FOUND"
    mask_mapped_id_found = test_df['MAPPED_BLINKIT_PRODUCT_ID'] != "NOT FOUND"

    # Calculate the lower and upper bounds for the acceptable price range (20% range)
    lower_bound = test_df['MRP'] * 0.8
    upper_bound = test_df['MRP'] * 1.2

    # Create a mask for valid BLINKIT_MRP values that are not null and where MAPPED_BLINKIT_PRODUCT_ID is found
    valid_blinkit_mask = test_df['BLINKIT_MRP'].notna() & mask_mapped_id_found

    # Update MAPPED_BLINKIT_PRODUCT_ID based on whether BLINKIT_MRP is within the acceptable range
    for idx in test_df[valid_blinkit_mask].index:
        if (test_df.at[idx, 'BLINKIT_MRP'] < lower_bound.at[idx]) or (test_df.at[idx, 'BLINKIT_MRP'] > upper_bound.at[idx]):
            test_df.at[idx, 'MAPPED_BLINKIT_PRODUCT_ID'] = "NOT FOUND"

    # Drop the BLINKIT_MRP column as it is no longer needed
    test_df.drop('BLINKIT_MRP', axis=1, inplace=True)

    return test_df

# Handling matches where MRP related discrepancy exists
zepto_df = return_mrp_vals_handled_test_df(original_df=blinkit_df, test_df=zepto_df)
instamart_df = return_mrp_vals_handled_test_df(original_df=blinkit_df, test_df=instamart_df)

### <font color='green'>**Saving the mapped results across Zepto and Instamart Platforms:**

#### <font color='purple'>**Transferring mapped blinkit product id's to the original test dataframes (Zepto and Instamart)**


In [ ]:
# For Saving the mapped results across Zepto and Instamart Platforms: Transferring mapped blinkit product id's to original dataframes (Zepto and Instamart)

# Platform: Zepto
save_zepto_df = zepto_df_orig.merge(zepto_df.loc[:,['PRODUCT_ID', 'MAPPED_BLINKIT_PRODUCT_ID']],
                left_on='product_id', right_on='PRODUCT_ID',
                how='inner')
save_zepto_df = save_zepto_df.rename(columns={'MAPPED_BLINKIT_PRODUCT_ID': 'mapped_blinkit_product_id'})\
                .drop('PRODUCT_ID', axis=1)
print(save_zepto_df.shape)
# Overwriting existing dataframe
zepto_df = save_zepto_df.copy()

# Platform: Instamart
save_instamart_df = instamart_df_orig.merge(instamart_df.loc[:,['PRODUCT_ID', 'MAPPED_BLINKIT_PRODUCT_ID']],
                left_on='product_id', right_on='PRODUCT_ID',
                how='inner')
save_instamart_df = save_instamart_df.rename(columns={'MAPPED_BLINKIT_PRODUCT_ID': 'mapped_blinkit_product_id'})\
                .drop('PRODUCT_ID', axis=1)
print(save_instamart_df.shape)

# Overwriting existing dataframe
instamart_df = save_instamart_df.copy()

(2626, 8)
(2400, 8)


### <font color='green'>**RESULTS**

<font color='purple'>**Validating a few Blinkit product recommendations implemented for Zepto products.**

In [ ]:
zepto_df.head(10)

,gc_platform,product_id,Category,brand,title,grammage,mrp,mapped_blinkit_product_id
0,Zepto,850aab83-b344-4dc5-a476-a8d6d6945021,"Masalas, Oils & Sugar",Daily Good,Daily Good Edible Gum / Dink,200 g,101.0,NOT FOUND
1,Zepto,85988d43-4312-4b77-b734-2ff60e6124e0,"Masalas, Oils & Sugar",Badshah,Badshah Kamal Tea Masala,100 g,132.0,NOT FOUND
2,Zepto,1bc33474-6b77-4028-a161-17f436636e53,"Biscuits, Cookies & Cakes",Britannia,Britannia 50-50 Potazos Masti Masala Crisps 71...,71.5 g X 2,50.0,NOT FOUND
3,Zepto,857b2613-87f3-4d3c-a26f-2a7fbac6e6a2,"Masalas, Oils & Sugar",Catch,Catch Jeera Whole,100 g,85.0,NOT FOUND
4,Zepto,b420ba0c-308f-4ed4-93ff-6ae2c8b59ce6,"Biscuits, Cookies & Cakes",Orion,Orion Strawberry Choco Pie,168 g,90.0,NOT FOUND
5,Zepto,2658b85a-67c5-45bc-a72b-a0d11abd2f7b,"Masalas, Oils & Sugar",Sunpure,Sunpure Sugar Pouch Bag,5 kg,295.0,NOT FOUND
6,Zepto,2661971d-efb2-44ff-97d8-a190d0bf6b10,"Biscuits, Cookies & Cakes",BUNDLE,Winkies Fruit Cake Raisin & Tutti Frutti 130 g...,130 g X 3,179.0,NOT FOUND
7,Zepto,03312228-a320-423c-a121-b8019725cade,"Biscuits, Cookies & Cakes",Unibic,Unibic Biscot Cinnamon Atta Biscuit,250 g,122.0,NOT FOUND
8,Zepto,0334390c-5dda-44bf-a3ed-96a866a25b04,"Biscuits, Cookies & Cakes",Parle,Parle Hide & Seek Milano Choco Filled Vanilla,125 g,75.0,NOT FOUND
9,Zepto,b0f429d3-8cc2-46c9-ba0b-cc7317e1b446,Ice Cream & Frozen Desserts,Mama Mia!,Mama Mia Blueberry Cheesecake Ice Cream,125 ml,159.0,546666


In [ ]:
# Proportion of NOT FOUND products on Zepto
zepto_not_found_count = zepto_df.loc[zepto_df['mapped_blinkit_product_id']=='NOT FOUND',:].shape[0]
print("\033[1m"+f"Percentage of products for which a match was found (Platform: Zepto): {round(100*(1 - (zepto_not_found_count/zepto_df.shape[0])),3)}%"+"\033[0m")

Percentage of products for which a match was found (Platform: Zepto): 13.823%


In [ ]:
zepto_df.isnull().sum()

,0
gc_platform,0
product_id,0
Category,0
brand,1
title,0
grammage,0
mrp,41
mapped_blinkit_product_id,0


In [ ]:
## Displaying original and mapped blinkit products for first few indices of Zepto dataset

# Pre-fetch columns once for Zepto DataFrame
zepto_columns = ['title', 'brand', 'Category', 'grammage', 'mrp', 'mapped_blinkit_product_id']
zepto_df_subset = zepto_df[zepto_columns].iloc[:100]

# Pre-fetch columns once for Blinkit DataFrame
blinkit_columns = ['product_id', 'title', 'brand', 'Category', 'grammage', 'mrp']
blinkit_df_subset = blinkit_df_orig[blinkit_columns]

sr_no = 0
for idx, row in zepto_df_subset.iterrows():
    zepto_product_title = row['title']
    zepto_product_brand = row['brand']
    zepto_product_category = row['Category']
    zepto_product_vol_size = row['grammage']
    zepto_product_mrp = row['mrp']

    mapped_blinkit_id = row['mapped_blinkit_product_id']

    if mapped_blinkit_id != 'NOT FOUND':
        sr_no += 1

        # Fetch the corresponding Blinkit product data in a single call
        blinkit_product = blinkit_df_subset[blinkit_df_subset['product_id'] == mapped_blinkit_id].iloc[0]

        mapped_blinkit_prod_title = blinkit_product['title']
        mapped_blinkit_prod_brand = blinkit_product['brand']
        mapped_blinkit_prod_category = blinkit_product['Category']
        mapped_blinkit_prod_vol_size = blinkit_product['grammage']
        mapped_blinkit_prod_mrp = blinkit_product['mrp']

        # Print the results
        print("\033[1m" + f"Product: {sr_no}" + "\033[0m");print()
        print(f"Zepto Product Name              : '{zepto_product_title}'")
        print(f"Product Brand                   : '{zepto_product_brand}'")
        print(f"Product Category                : '{zepto_product_category}'")
        print(f"Product Grammage                : '{zepto_product_vol_size}'")
        print(f"Product MRP                     : '{zepto_product_mrp}'");print()

        print(f"Recommended Blinkit Product Name: '{mapped_blinkit_prod_title}'")
        print(f"Product Brand                   : '{mapped_blinkit_prod_brand}'")
        print(f"Product Category                : '{mapped_blinkit_prod_category}'")
        print(f"Product Grammage                : '{mapped_blinkit_prod_vol_size}'")
        print(f"Product MRP                     : '{mapped_blinkit_prod_mrp}'");print("\n")


Product: 1

Zepto Product Name              : 'Mama Mia Blueberry Cheesecake Ice Cream'
Product Brand                   : 'Mama Mia!'
Product Category                : 'Ice Cream & Frozen Desserts'
Product Grammage                : '125 ml'
Product MRP                     : '159.0'

Recommended Blinkit Product Name: 'Mama Mia Blueberry Cheesecake Ice Cream Cup'
Product Brand                   : 'Mama Mia'
Product Category                : 'Ice Cream & Frozen Desserts'
Product Grammage                : '100 g'
Product MRP                     : '159.0'


Product: 2

Zepto Product Name              : 'Parry's Superfine Sugar'
Product Brand                   : 'Parry's'
Product Category                : 'Masalas, Oils & Sugar'
Product Grammage                : '1 kg'
Product MRP                     : '75.0'

Recommended Blinkit Product Name: 'Parry's Superfine Sugar'
Product Brand                   : 'Parry's'
Product Category                : 'Masalas, Oils & Sugar'
Product Grammage      

<font color='purple'>**Validating a few Blinkit product recommendations implemented for Instamart products.**

In [ ]:
instamart_df.head(10)

,gc_platform,product_id,Category,brand,title,grammage,mrp,mapped_blinkit_product_id
0,Instamart,1KMX6G1RR5,"Masalas, Oils & Sugar",Parry's,Parry's Sweet Care Low GI Sugar,500 g,60.0,449049
1,Instamart,PBYMNPG7SJ,Ice Cream & Frozen Desserts,Heritage,Heritage Alpenvie Caramel Nuts Cone,110 ml,40.0,NOT FOUND
2,Instamart,HVHM3V7ON9,Ice Cream & Frozen Desserts,Papacream,Papacream Hazelnut Cold Coffee Ice Cream Cup,100 ml,132.0,NOT FOUND
3,Instamart,HXU8O1FGXT,"Masalas, Oils & Sugar",Maggi,Maggi Masala-ae-Magic,72 g,60.0,NOT FOUND
4,Instamart,LMB1DP2R02,"Biscuits, Cookies & Cakes",Walkers,Walkers Belgian Chocolate Chunk Biscuits,150 g,440.0,114471
5,Instamart,LOGVGV745L,"Biscuits, Cookies & Cakes",Unibic,Unibic Cashew Finger Cookies,200 g,199.0,NOT FOUND
6,Instamart,BCX6GXPKAR,"Masalas, Oils & Sugar",Chukde,Chukde Jeera Sabut,100 g,72.0,NOT FOUND
7,Instamart,BKUKX4DMDN,"Biscuits, Cookies & Cakes",Parle,Parle Hide & Seek Cafe Mocha Cookies,100 g,30.0,NOT FOUND
8,Instamart,80TMBLMXFD,"Masalas, Oils & Sugar",Maya,Maya Whole Spice Clove,25 g,50.0,NOT FOUND
9,Instamart,8XBD4RW2RO,"Masalas, Oils & Sugar",SHYAM,SHYAM Bay Leaf (Tej Patta),50 g,20.0,521892


In [ ]:
# Proportion of NOT FOUND products on Instamart
instamart_not_found_count = instamart_df.loc[instamart_df['mapped_blinkit_product_id']=='NOT FOUND',:].shape[0]
print("\033[1m"+f"Percentage of products for which a match was found (Platform: Instamart): {round(100*(1 - (instamart_not_found_count/instamart_df.shape[0])),2)}%"+"\033[0m")

Percentage of products for which a match was found (Platform: Instamart): 15.21%


In [ ]:
## Displaying original and mapped blinkit products for first few indices of Instamart dataset

# Pre-fetch necessary columns from Instamart DataFrame
instamart_columns = ['title', 'brand', 'Category', 'grammage', 'mrp', 'mapped_blinkit_product_id']
instamart_df_subset = instamart_df[instamart_columns].iloc[:100]

# Pre-fetch necessary columns from Blinkit DataFrame
blinkit_columns = ['product_id', 'title', 'brand', 'Category', 'grammage', 'mrp']
blinkit_df_subset = blinkit_df_orig[blinkit_columns]

sr_no = 0
for idx, row in instamart_df_subset.iterrows():
    instamart_product_title = row['title']
    instamart_product_brand = row['brand']
    instamart_product_category = row['Category']
    instamart_product_vol_size = row['grammage']
    instamart_product_mrp = row['mrp']

    mapped_blinkit_id = row['mapped_blinkit_product_id']

    if mapped_blinkit_id != 'NOT FOUND':
        sr_no += 1

        # Fetch the corresponding Blinkit product data in a single call
        blinkit_product = blinkit_df_subset[blinkit_df_subset['product_id'] == mapped_blinkit_id].iloc[0]

        mapped_blinkit_prod_title = blinkit_product['title']
        mapped_blinkit_prod_brand = blinkit_product['brand']
        mapped_blinkit_prod_category = blinkit_product['Category']
        mapped_blinkit_prod_vol_size = blinkit_product['grammage']
        mapped_blinkit_prod_mrp = blinkit_product['mrp']

        # Print the results
        print("\033[1m" + f"Product: {sr_no}" + "\033[0m");print()
        print(f"Instamart Product Name              : '{instamart_product_title}'")
        print(f"Product Brand                       : '{instamart_product_brand}'")
        print(f"Product Category                    : '{instamart_product_category}'")
        print(f"Product Grammage                    : '{instamart_product_vol_size}'")
        print(f"Product MRP                         : '{instamart_product_mrp}'");print()

        print(f"Recommended Blinkit Product Name    : '{mapped_blinkit_prod_title}'")
        print(f"Product Brand                       : '{mapped_blinkit_prod_brand}'")
        print(f"Product Category                    : '{mapped_blinkit_prod_category}'")
        print(f"Product Grammage                    : '{mapped_blinkit_prod_vol_size}'")
        print(f"Product MRP                         : '{mapped_blinkit_prod_mrp}'");print("\n")

Product: 1

Instamart Product Name              : 'Parry's Sweet Care Low GI Sugar'
Product Brand                       : 'Parry's'
Product Category                    : 'Masalas, Oils & Sugar'
Product Grammage                    : '500 g'
Product MRP                         : '60.0'

Recommended Blinkit Product Name    : 'Parry's Sweet Care Low GI Sugar'
Product Brand                       : 'Parry's'
Product Category                    : 'Masalas, Oils & Sugar'
Product Grammage                    : '500 g'
Product MRP                         : '60.0'


Product: 2

Instamart Product Name              : 'Walkers Belgian Chocolate Chunk Biscuits'
Product Brand                       : 'Walkers'
Product Category                    : 'Biscuits, Cookies & Cakes'
Product Grammage                    : '150 g'
Product MRP                         : '440.0'

Recommended Blinkit Product Name    : 'Walkers Belgian Chocolate Chunk Biscuit'
Product Brand                       : 'Walkers'
Product Cat

<font color='green'>**Saving the mapped results across Zepto and Instamart Platforms**

# <font color='purple'>**DESIRED OUTPUT**

In [ ]:
# Size of original dataframes
print(blinkit_df_orig.shape)
print(zepto_df_orig.shape)
print(instamart_df_orig.shape)

(3162, 7)
(2626, 7)
(2400, 7)


### <font color='blue'>**Original Blinkit Dataset** *(Relevant columns)*

In [ ]:
import os

# -----------------------------------------------------------------------------------------
# SAVING THE MAPPED RESULTS ACROSS ZEPTO AND INSTAMART PLATFORMS
# -----------------------------------------------------------------------------------------

# Filepath to save Zepto and Instamart Datasets (with Mapped Blinkit Product IDs)
save_zepto_df_filepath = "/content/drive/MyDrive/Results/Zepto.csv"
save_instamart_df_filepath = "/content/drive/MyDrive/Results/Instamart.csv"

# Get the directory path from the file paths
zepto_dir = os.path.dirname(save_zepto_df_filepath)
instamart_dir = os.path.dirname(save_instamart_df_filepath)

# Check if the directories exist, if not create them
os.makedirs(zepto_dir, exist_ok=True)
os.makedirs(instamart_dir, exist_ok=True)

# Saving as a CSV file
zepto_df.to_csv(save_zepto_df_filepath, index=False)
instamart_df.to_csv(save_instamart_df_filepath, index=False)


In [ ]:
# Extracting relevant columns from Blinkit original df
blinkit_df_orig_relevant = blinkit_df_orig.rename(columns={"title": "Blinkit Product Name", "product_id": "Blinkit Product ID"})
print(blinkit_df_orig_relevant.shape)
blinkit_df_orig_relevant.head(10)

(3162, 7)


,gc_platform,Blinkit Product ID,Category,brand,Blinkit Product Name,grammage,mrp
0,Blinkit,185986,"Masalas, Oils & Sugar",Tata Sampann,Tata Sampann Dal Tadka Masala with Natural Oils,100 g,85.0
1,Blinkit,111953,"Masalas, Oils & Sugar",Sakthi,Sakthi Coriander Powder/Dhania,200 g,64.0
2,Blinkit,362861,"Biscuits, Cookies & Cakes",Karachi Bakery,"Karachi Bakery Triple Delight Fruit, Cashew & ...",600 g,329.0
3,Blinkit,116568,"Biscuits, Cookies & Cakes",Malpani's Bake-lite,Malpani's Bake-lite Puneri Special Cream Roll,180 g (5 pieces),69.0
4,Blinkit,25278,"Masalas, Oils & Sugar",Everest,Everest Royal Garam Masala,100 g,101.0
5,Blinkit,495432,"Masalas, Oils & Sugar",Parry's,Parry's Superfine Sugar,1 kg,75.0
6,Blinkit,495860,Ice Cream & Frozen Desserts,Baskin Robbins,Baskin Robbins Cookies n Cream Ice Cream Cone,110 ml,88.0
7,Blinkit,378787,"Masalas, Oils & Sugar",Katdare,Katdare Byadgi Chilli Powder,200 g,154.0
8,Blinkit,101606,"Biscuits, Cookies & Cakes",Bisk Farm,Bisk Farm Crispy Cream Cracker Biscuit,250 g,45.0
9,Blinkit,108085,"Biscuits, Cookies & Cakes",Britannia,Britannia Fruit Cake (Veg),60 g,25.0


## <font color='maroon'>**NOTE:**<br>
**This dataset might contain rare cases with same Blinkit product mapped across multiple Zepto or Instamart products. Thus, creating multiple records in the original Blinkit Dataset when merged. Dealing with such cases...**

### <font color='purple'>**Dealing with such cases**

### <font color='purple'>**(i) Integration ->  Blinkit + Mapped Zepto Products**

In [ ]:
# Zepto df (mapped dataframe) -> Considering only all those cases where Blinkit Product ID was found
zepto_existing_df = zepto_df.loc[(~pd.isna(zepto_df['mapped_blinkit_product_id']) & \
                                 (zepto_df['mapped_blinkit_product_id'] != "NOT FOUND")),:].drop_duplicates()
zepto_existing_df = zepto_existing_df.loc[:, ['product_id', 'mapped_blinkit_product_id', 'title']]\
                            .rename(columns={"product_id": "Zepto Product ID", \
                                             "mapped_blinkit_product_id": "Blinkit Product ID",
                                             "title": "Zepto Product Name"})

# Mapping aligning Zepto product IDs to Blinkit product IDs
merge_blinkit_zepto_df = pd.merge(blinkit_df_orig_relevant, zepto_existing_df,
                        on = "Blinkit Product ID", how="left").fillna({'Zepto Product ID': "NOT FOUND"})



import pandas as pd
import Levenshtein

def reorder_column_based_on_another(df1, df2, col_df1, col_df2):
    """
    Reorders the column in df2 based on the values and order in df1.

    Parameters:
    df1 (pd.DataFrame): The first DataFrame whose column determines the order.
    df2 (pd.DataFrame): The second DataFrame whose column will be reordered.
    col_df1 (str): The column in df1 used to determine the order.
    col_df2 (str): The column in df2 to be reordered.

    Returns:
    pd.DataFrame: The df2 with reordered column based on df1's order.
    """
    # Create a mapping of values in df1[col_df1] to their positions
    order_map = {value: idx for idx, value in enumerate(df1[col_df1])}

    # Reorder df2[col_df2] based on the order of values in df1[col_df1]
    df2_sorted = df2.loc[df2[col_df2].map(order_map).sort_values().index].reset_index(drop=True)

    return df2_sorted


def filter_by_levenshtein_similarity(df, original_df, id_col, name_col_1, name_col_2):
    """
    Filters a DataFrame to retain rows with duplicated values in a specified ID column.
    For each group of duplicated IDs, it selects the row with the smallest Levenshtein
    distance between two name columns. The resulting DataFrame is then reordered
    according to the order of IDs in the `original_df`.

    Parameters:
        df (pd.DataFrame): The DataFrame to be filtered and processed.
        original_df (pd.DataFrame): The reference DataFrame used to maintain the order of IDs.
        id_col (str): The name of the column containing the ID values to check for duplication (e.g., 'Blinkit Product ID').
        name_col_1 (str): The first name column used for Levenshtein distance comparison (e.g., 'Blinkit Product Name').
        name_col_2 (str): The second name column used for Levenshtein distance comparison (e.g., 'Zepto Product Name').

    Returns:
        pd.DataFrame: A DataFrame filtered based on the smallest Levenshtein distance for duplicated IDs,
                      and reordered to match the order of IDs in `original_df`.

    Raises:
        ValueError: If the number of unique IDs in the resulting DataFrame does not match the
                    number of products in the `original_df`, indicating a mapping error.
    """
    print("Processing...")
    # Ensure the name columns are strings
    df[name_col_1] = df[name_col_1].fillna("").astype(str)
    df[name_col_2] = df[name_col_2].fillna("").astype(str)

    # Step 1: Identify repeated IDs
    repeated_ids = df[id_col].value_counts()
    repeated_ids = repeated_ids[repeated_ids > 1].index

    # Step 2: Filter rows with repeated IDs
    filtered_df = df[df[id_col].isin(repeated_ids)].copy()

    # Step 3: Calculate Levenshtein distance
    filtered_df["Levenshtein Distance"] = filtered_df.apply(
        lambda row: Levenshtein.distance(row[name_col_1], row[name_col_2]), axis=1
    )

    # Step 4: Retain rows with the smallest Levenshtein distance
    filtered_df = filtered_df.loc[
        filtered_df.groupby(id_col)["Levenshtein Distance"].idxmin()
    ]

    # Step 5: Combine with non-duplicated IDs
    result_df = pd.concat([df[~df[id_col].isin(repeated_ids)], filtered_df])

    # Drop the Levenshtein Distance column for the final output
    result_df = result_df.drop(columns=["Levenshtein Distance"])

    # # Step 6: Sort result_df based on the order of 'product_id' in original_df
    result_df = reorder_column_based_on_another(df1=original_df,
                                    df2=result_df,
                                    col_df1='product_id',
                                    col_df2=id_col)
    # Checking if every product has one single mapping
    if original_df.shape[0] != result_df[id_col].nunique():
        raise ValueError(f"Mapping error: The number of unique IDs in 'result' ({result_df[id_col].nunique()}) does not match the number of products in 'original_df' ({original_df.shape[0]}).")
    else:
        print("Single mappings generated across every product ID. Validation completed successfully!")

    return result_df


# In case, if the record count is not same, meaning multiple mappings are generated
# Handling this discrepancy

print(f"\nFor Mapping 'Blinkit' vs 'Zepto' Products:")
if blinkit_df_orig.shape[0] != merge_blinkit_zepto_df.shape[0]:
  print("Multiple mappings are generated. Dealing with such cases.")
  merge_blinkit_zepto_df = filter_by_levenshtein_similarity(df=merge_blinkit_zepto_df,
                                                            original_df=blinkit_df_orig,
                                                            id_col="Blinkit Product ID",
                                                            name_col_1="Blinkit Product Name",
                                                            name_col_2="Zepto Product Name")
else:
  print("Single mappings found across every product ID in the Blinkit dataset.")


# Retaining relevant columns
print("Merging Blinkit and Zepto Results...")
final_blinkit_zepto_merge = merge_blinkit_zepto_df.loc[:, ['Blinkit Product Name', 'Blinkit Product ID', 'Zepto Product ID']]
print("Merge complete!\n")



For Mapping 'Blinkit' vs 'Zepto' Products:
Multiple mappings are generated. Dealing with such cases.
Processing...
Single mappings generated across every product ID. Validation completed successfully!
Merging Blinkit and Zepto Results...
Merge complete!



In [ ]:
final_blinkit_zepto_merge.head(5)

,Blinkit Product Name,Blinkit Product ID,Zepto Product ID
0,Tata Sampann Dal Tadka Masala with Natural Oils,185986,NOT FOUND
1,Sakthi Coriander Powder/Dhania,111953,NOT FOUND
2,"Karachi Bakery Triple Delight Fruit, Cashew & ...",362861,NOT FOUND
3,Malpani's Bake-lite Puneri Special Cream Roll,116568,NOT FOUND
4,Everest Royal Garam Masala,25278,edae08a4-9223-4d14-8231-309573034a13


### <font color='purple'>**(ii) Integration ->  Blinkit + Mapped Instamart Products**

In [ ]:
# Instamart df (mapped dataframe) -> Considering only all those cases where Blinkit Product ID was found
instamart_existing_df = instamart_df.loc[(~pd.isna(instamart_df['mapped_blinkit_product_id']) & \
                                 (instamart_df['mapped_blinkit_product_id'] != "NOT FOUND")),:].drop_duplicates()
instamart_existing_df = instamart_existing_df.loc[:, ['product_id', 'mapped_blinkit_product_id', 'title']]\
                            .rename(columns={"product_id": "Instamart Product ID", \
                                             "mapped_blinkit_product_id": "Blinkit Product ID",
                                             "title": "Instamart Product Name"})

# Mapping aligning Instamart product IDs to Blinkit product IDs
merge_blinkit_instamart_df = pd.merge(blinkit_df_orig_relevant, instamart_existing_df,
                        on = "Blinkit Product ID", how="left").fillna({'Instamart Product ID': "NOT FOUND"})



# In case, if the record count is not same, meaning multiple mappings are generated
# Handling this discrepancy

print(f"\nFor Mapping 'Blinkit' vs 'Instamart' Products:")
if blinkit_df_orig.shape[0] != merge_blinkit_instamart_df.shape[0]:
  print("Multiple mappings are generated. Dealing with such cases.")
  merge_blinkit_instamart_df = filter_by_levenshtein_similarity(df=merge_blinkit_instamart_df,
                                                            original_df=blinkit_df_orig,
                                                            id_col="Blinkit Product ID",
                                                            name_col_1="Blinkit Product Name",
                                                            name_col_2="Instamart Product Name")
else:
  print("Single mappings found across every product ID in the Blinkit dataset.")


# Retaining relevant columns
print("Merging Blinkit and Instamart Results...")
final_blinkit_instamart_merge = merge_blinkit_instamart_df.loc[:, ['Blinkit Product Name', 'Blinkit Product ID', 'Instamart Product ID']]
print("Merge complete!\n")


For Mapping 'Blinkit' vs 'Instamart' Products:
Multiple mappings are generated. Dealing with such cases.
Processing...
Single mappings generated across every product ID. Validation completed successfully!
Merging Blinkit and Instamart Results...
Merge complete!



In [ ]:
final_blinkit_instamart_merge.head(5)

,Blinkit Product Name,Blinkit Product ID,Instamart Product ID
0,Tata Sampann Dal Tadka Masala with Natural Oils,185986,KCZWPNFTMN
1,Sakthi Coriander Powder/Dhania,111953,NOT FOUND
2,"Karachi Bakery Triple Delight Fruit, Cashew & ...",362861,NOT FOUND
3,Malpani's Bake-lite Puneri Special Cream Roll,116568,NOT FOUND
4,Everest Royal Garam Masala,25278,NOT FOUND


In [ ]:
# Final Results
final_results_data = pd.merge(final_blinkit_zepto_merge,
                              final_blinkit_instamart_merge,
                              on=["Blinkit Product Name", "Blinkit Product ID"],
                              how="inner")
final_results_data.head(20)

,Blinkit Product Name,Blinkit Product ID,Zepto Product ID,Instamart Product ID
0,Tata Sampann Dal Tadka Masala with Natural Oils,185986,NOT FOUND,KCZWPNFTMN
1,Sakthi Coriander Powder/Dhania,111953,NOT FOUND,NOT FOUND
2,"Karachi Bakery Triple Delight Fruit, Cashew & ...",362861,NOT FOUND,NOT FOUND
3,Malpani's Bake-lite Puneri Special Cream Roll,116568,NOT FOUND,NOT FOUND
4,Everest Royal Garam Masala,25278,edae08a4-9223-4d14-8231-309573034a13,NOT FOUND
5,Parry's Superfine Sugar,495432,479486c7-ac6b-4d9c-bce8-599196739365,NOT FOUND
6,Baskin Robbins Cookies n Cream Ice Cream Cone,495860,7e9ce602-2289-40db-b62f-cf876718ac6e,NOT FOUND
7,Katdare Byadgi Chilli Powder,378787,NOT FOUND,NOT FOUND
8,Bisk Farm Crispy Cream Cracker Biscuit,101606,NOT FOUND,NOT FOUND
9,Britannia Fruit Cake (Veg),108085,NOT FOUND,NOT FOUND


In [ ]:
# Proportion of NOT FOUND products on Zepto
zepto_not_found_count_final = final_results_data.loc[final_results_data['Zepto Product ID']=='NOT FOUND',:].shape[0]
# Percentage of Blinkit products for which a match was found (Platform: Zepto)
print("\033[1m"+f"Percentage of Blinkit products for which an exact match was found (Platform: Zepto)    : {round(100*(1 - (zepto_not_found_count_final/final_results_data.shape[0])),2)}%"+"\033[0m")

# Proportion of NOT FOUND products on Instamart
instamart_not_found_count_final = final_results_data.loc[final_results_data['Instamart Product ID']=='NOT FOUND',:].shape[0]
# Percentage of Blinkit products for which a match was found (Platform: Instamart)
print("\033[1m"+f"Percentage of Blinkit products for which an exact match was found (Platform: Instamart): {round(100*(1 - (instamart_not_found_count_final/final_results_data.shape[0])),2)}%"+"\033[0m")


Percentage of Blinkit products for which an exact match was found (Platform: Zepto)    : 11.45%
Percentage of Blinkit products for which an exact match was found (Platform: Instamart): 11.48%


### <font color='purple'>**Saving the final results table**

In [ ]:
# Saving the final results data containing Blinkit products mapped to their respective matches on Zepto and Instamart
final_results_data.to_csv("/content/drive/MyDrive/Results/Output.csv", index=False)